## Extracting Smartfin Ride Time

### Import necessary libraries:

In [8]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt

plt.rc("font", size=14) 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#from mpl_toolkits.basemap import Basemap


import os
import datetime
import pytz
import re

import peakutils
import statsmodels.api as sm

import requests

#Read data from a local csv file:

##Will change this to scrape files from the Smartfin.org website later.
#data = pd.read_csv('Motion_13735.CSV', header=0)   
#data = data.dropna()

#Print out the column headings:
#print(data.shape)
#print(list(data.columns))

## Fin ID Scraper (pulls dataframes for specific ride id from website):

In [9]:
#%% Fin ID scraper
# Input fin ID, get all ride IDs
# base URL to which we'll append given fin IDs
fin_url_base = 'http://surf.smartfin.org/fin/'

# Look for the following text in the HTML contents in fcn below
str_id_ride = 'rideId = \'' # backslash allows us to look for single quote
str_id_date = 'var date = \'' # backslash allows us to look for single quote

#%% Ride ID scraper
# Input ride ID, get ocean and motion CSVs
# Base URL to which we'll append given ride IDs
ride_url_base = 'https://surf.smartfin.org/ride/'

# Look for the following text in the HTML contents in fcn below
str_id_csv = 'img id="temperatureChart" class="chart" src="' 




def get_csv_from_ride_id(self, rid):
    # Build URL for each individual ride
    ride_url = ride_url_base+str(rid)
    print(ride_url)

    # Get contents of ride_url
    html_contents = requests.get(ride_url).text

    # Find CSV identifier 
    loc_csv_id = html_contents.find(str_id_csv)

    # Different based on whether user logged in with FB or Google
    offset_googleOAuth = [46, 114]
    offset_facebkOAuth = [46, 112]
    if html_contents[loc_csv_id+59] == 'f': # Facebook login
        off0 = offset_facebkOAuth[0]
        off1 = offset_facebkOAuth[1]
    else: # Google login
        off0 = offset_googleOAuth[0]
        off1 = offset_googleOAuth[1]

    csv_id_longstr = html_contents[loc_csv_id+off0:loc_csv_id+off1]

#    print(csv_id_longstr)

    # Stitch together full URL for CSV
    if ("media" in csv_id_longstr) & ("Calibration" not in html_contents): # other junk URLs can exist and break everything

        ocean_csv_url = 'https://surf.smartfin.org/'+csv_id_longstr+'Ocean.CSV'
        motion_csv_url = 'https://surf.smartfin.org/'+csv_id_longstr+'Motion.CSV'

        print(ocean_csv_url)
        # Go to ocean_csv_url and grab contents (theoretically, a CSV)
        ocean_df_small = pd.read_csv(ocean_csv_url, parse_dates = [0])
        elapsed_timedelta = (ocean_df_small['UTC']-ocean_df_small['UTC'][0])
        ocean_df_small['elapsed'] = elapsed_timedelta/np.timedelta64(1, 's')

        motion_df_small = pd.read_csv(motion_csv_url, parse_dates = [0])

        # Reindex on timestamp if there are at least a few rows
        if len(ocean_df_small) > 1:
            ocean_df_small.set_index('UTC', drop = True, append = False, inplace = True)
            motion_df_small.set_index('UTC', drop = True, append = False, inplace = True)

            #print(ocean_df_small)
            #print(motion_df_small)

            #May need to change this sampling interval:
            sample_interval = '33ms'


            ocean_df_small_resample = ocean_df_small.resample(sample_interval).mean()
            motion_df_small_resample = motion_df_small.resample(sample_interval).mean()

            # No need to save many extra rows with no fix
            motion_df_small = motion_df_small[~np.isnan(motion_df_small.Latitude)]

            return ocean_df_small_resample, motion_df_small_resample

    else:
        ocean_df_small_resample = pd.DataFrame() # empty DF just so something is returned
        motion_df_small_resample = pd.DataFrame() 
        return ocean_df_small_resample, motion_df_small_resample

## Build dataframe from ride id

In [10]:

appended_ocean_list = [] # list of DataFrames from original CSVs
appended_motion_list = []
appended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame

## Nested loops (for each fin ID, find all ride IDs, then build a DataFrame from all ride CSVs)
## (Here, ride IDS are either ocean or motion dataframes)
count_good_fins = 0

# Loop over ride_ids and find CSVs
try:
    new_ocean_df, new_motion_df = get_csv_from_ride_id(ride_id) # get given ride's CSV from its ride ID using function above
    print(len(new_ocean_df))
    print(len(new_motion_df))
    if not new_ocean_df.empty: # Calibration rides, for example
        # Append only if DF isn't empty. There may be a better way to control empty DFs which are created above
        appended_multiIndex.append(str(rid)) # build list to be multiIndex of future DataFrame
        appended_ocean_list.append(new_ocean_df)
        appended_motion_list.append(new_motion_df)
        print("Ride data has been uploaded.")
        #print("Ride: ", rid, "data has been uploaded.")
        count_good_fins += 1

except: 
    print("Ride threw an exception!")
    #print("Ride ", rid, "threw an exception!")    


df_keys = tuple(appended_multiIndex) # keys gotta be a tuple, a list which data in it cannot be changed

motion_df = pd.concat(appended_motion_list, keys = df_keys, names = ['ride_id'])

Ride threw an exception!


ValueError: No objects to concatenate

## Drop the NA values from the dataframe:

In [23]:
#Drop the latitude and longitude values since most of them are Nan:
motion_df_dropped = motion_df.drop(columns=['Latitude', 'Longitude'])


#Drop the NAN values from the motion data:
motion_df_dropped = motion_df_dropped.dropna(axis=0, how='any')
print('motion_df_dropped length: ', len(motion_df_dropped))
motion_df_dropped.head(10)

motion_df_dropped length:  21645


Time  IMU A1  IMU A2  \
ride_id UTC                                                              
15692   2018-11-09 19:16:03.789000+00:00  1.414743e+09   493.0    48.0   
        2018-11-09 19:16:04.053000+00:00  1.414743e+09   513.0    89.0   
        2018-11-09 19:16:04.284000+00:00  1.414743e+09   494.0    92.0   
        2018-11-09 19:16:04.548000+00:00  1.414744e+09   421.0   205.0   
        2018-11-09 19:16:04.812000+00:00  1.414744e+09   534.0   306.0   
        2018-11-09 19:16:05.043000+00:00  1.414744e+09   455.0   149.0   
        2018-11-09 19:16:05.307000+00:00  1.414744e+09   474.0   342.0   
        2018-11-09 19:16:05.571000+00:00  1.414745e+09   363.0   323.0   
        2018-11-09 19:16:05.802000+00:00  1.414745e+09   -21.0   510.0   
        2018-11-09 19:16:06.066000+00:00  1.414745e+09    35.0   283.0   

                                          IMU A3  IMU G1  IMU G2  IMU G3  \
ride_id UTC                                                                
15692   2018-11-09 19:16:03.789000+00:00   110.0    75.0  -124.0   -86.0   
        2018-11-09 19:16:04.053000+00:00    62.0    34.0   -36.0   -92.0   
        2018-11-09 19:16:04.284000+00:00    80.0    69.0   -63.0   -42.0   
        2018-11-09 19:16:04.548000+00:00  -104.0   192.0   -92.0   -37.0   
        2018-11-09 19:16:04.812000+00:00   -32.0  -421.0  -233.0  -229.0   
        2018-11-09 19:16:05.043000+00:00  -102.0  -355.0  -376.0  -397.0   
        2018-11-09 19:16:05.307000+00:00  -219.0  -234.0  -527.0  -465.0   
        2018-11-09 19:16:05.571000+00:00  -131.0    60.0  -662.0  -305.0   
        2018-11-09 19:16:05.802000+00:00  -447.0    78.0  -643.0  -153.0   
        2018-11-09 19:16:06.066000+00:00  -132.0  -114.0  -430.0   132.0   

                                          IMU M1  IMU M2  IMU M3  
ride_id UTC                                                       
15692   2018-11-09 19:16:03.789000+00:00  -309.0   209.0    39.0  
        2018-11-09 19:16:04.053000+00:00  -320.0   194.0    38.0  
        2018-11-09 19:16:04.284000+00:00  -329.0   189.0    49.0  
        2018-11-09 19:16:04.548000+00:00  -330.0   180.0    64.0  
        2018-11-09 19:16:04.812000+00:00  -325.0   161.0    97.0  
        2018-11-09 19:16:05.043000+00:00  -337.0   117.0   151.0  
        2018-11-09 19:16:05.307000+00:00  -311.0    25.0   217.0  
        2018-11-09 19:16:05.571000+00:00  -238.0    -8.0   272.0  
        2018-11-09 19:16:05.802000+00:00  -159.0   -21.0   321.0  
        2018-11-09 19:16:06.066000+00:00   -86.0   -38.0   326.0

## Remove ride_id from index

In [36]:
df = motion_df_dropped.reset_index()
df = df.drop('ride_id', axis=1)
df = df.set_index('UTC')
df.head(10)

,Time,IMU A1,IMU A2,IMU A3,IMU G1,IMU G2,IMU G3,IMU M1,IMU M2,IMU M3
UTC,,,,,,,,,,
2018-11-09 19:16:03.789000+00:00,1.414743e+09,493.0,48.0,110.0,75.0,-124.0,-86.0,-309.0,209.0,39.0
2018-11-09 19:16:04.053000+00:00,1.414743e+09,513.0,89.0,62.0,34.0,-36.0,-92.0,-320.0,194.0,38.0
2018-11-09 19:16:04.284000+00:00,1.414743e+09,494.0,92.0,80.0,69.0,-63.0,-42.0,-329.0,189.0,49.0
2018-11-09 19:16:04.548000+00:00,1.414744e+09,421.0,205.0,-104.0,192.0,-92.0,-37.0,-330.0,180.0,64.0
2018-11-09 19:16:04.812000+00:00,1.414744e+09,534.0,306.0,-32.0,-421.0,-233.0,-229.0,-325.0,161.0,97.0
2018-11-09 19:16:05.043000+00:00,1.414744e+09,455.0,149.0,-102.0,-355.0,-376.0,-397.0,-337.0,117.0,151.0
2018-11-09 19:16:05.307000+00:00,1.414744e+09,474.0,342.0,-219.0,-234.0,-527.0,-465.0,-311.0,25.0,217.0
2018-11-09 19:16:05.571000+00:00,1.414745e+09,363.0,323.0,-131.0,60.0,-662.0,-305.0,-238.0,-8.0,272.0
2018-11-09 19:16:05.802000+00:00,1.414745e+09,-21.0,510.0,-447.0,78.0,-643.0,-153.0,-159.0,-21.0,321.0


## Create an elapsed time field to sync Smartfin data with Video Footage:

In [37]:
#Create an elapsed_timedelta field:

#timedelta_values = (motion_df_dropped['Time']-motion_df_dropped['Time'][0])
#motion_df_dropped.insert(loc=1, column='TimeDelta', value=timedelta_values, drop=True)
df['TimeDelta'] = (df['Time']-df['Time'][0])
#print(elapsed_timedelta)
#motion_df_dropped.head()
df

,Time,IMU A1,IMU A2,IMU A3,IMU G1,IMU G2,IMU G3,IMU M1,IMU M2,IMU M3,TimeDelta
UTC,,,,,,,,,,,
2018-11-09 19:16:03.789000+00:00,1.414743e+09,493.0,48.0,110.0,75.0,-124.0,-86.0,-309.0,209.0,39.0,0.0
2018-11-09 19:16:04.053000+00:00,1.414743e+09,513.0,89.0,62.0,34.0,-36.0,-92.0,-320.0,194.0,38.0,252.5
2018-11-09 19:16:04.284000+00:00,1.414743e+09,494.0,92.0,80.0,69.0,-63.0,-42.0,-329.0,189.0,49.0,501.5
2018-11-09 19:16:04.548000+00:00,1.414744e+09,421.0,205.0,-104.0,192.0,-92.0,-37.0,-330.0,180.0,64.0,753.5
2018-11-09 19:16:04.812000+00:00,1.414744e+09,534.0,306.0,-32.0,-421.0,-233.0,-229.0,-325.0,161.0,97.0,1003.5
...,...,...,...,...,...,...,...,...,...,...,...
2018-11-09 20:38:14.055000+00:00,1.419643e+09,501.0,-11.0,99.0,9.0,21.0,2.0,-293.0,259.0,41.0,4900552.5
2018-11-09 20:38:14.319000+00:00,1.419644e+09,501.0,-11.0,99.0,9.0,20.0,2.0,-303.0,267.0,37.0,4900803.5
2018-11-09 20:38:14.583000+00:00,1.419644e+09,502.0,-11.0,99.0,10.0,21.0,1.0,-308.0,262.0,32.0,4901054.5


## Convert UTC Timestamps to UNIX 

In [21]:
df['Time'][0] 

1414742885.5

In [52]:
# get year
start_time = pd.to_datetime(df.index[0]).strftime('%d/%m/%Y %H:%M:%S')
end_time = pd.to_datetime(df.index[-1]).strftime('%d/%m/%Y %H:%M:%S')
year_date = start_time[6:10]

In [53]:
start_time

'09/11/2018 19:16:03'

In [54]:
end_time

'09/11/2018 20:38:15'

In [55]:
year_date

'2018'

In [43]:
import smartfin_web_scraper as sws

module

In [44]:
scraper = sws.web_scraper()


AttributeError: module 'smartfin_web_scraper' has no attribute 'web_scraper'